### [How to inspect runnables](https://python.langchain.com/docs/how_to/inspect/)

In [1]:
import getpass
import os

if "LANGCHAIN_API_KEY" not in os.environ:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [2]:
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

#### A. Chain

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["harrison worked at kensho"], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [5]:
chain.get_graph()

Graph(nodes={'30e5a545cbc542a886e45ab00e813e1e': Node(id='30e5a545cbc542a886e45ab00e813e1e', name='Parallel<context,question>Input', data=<class 'langchain_core.runnables.base.RunnableParallel<context,question>Input'>, metadata=None), 'd1aeef20726049838092505ae8448995': Node(id='d1aeef20726049838092505ae8448995', name='Parallel<context,question>Output', data=<class 'langchain_core.utils.pydantic.RunnableParallel<context,question>Output'>, metadata=None), '1f3e9076c2ca4936bd940a2566c88182': Node(id='1f3e9076c2ca4936bd940a2566c88182', name='VectorStoreRetriever', data=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x177baa4d0>, search_kwargs={}), metadata=None), '37a8b26250cf41c6829e8e92f4c5604b': Node(id='37a8b26250cf41c6829e8e92f4c5604b', name='Passthrough', data=RunnablePassthrough(), metadata=None), '4f41348daa2746c681394e844ab514a1': Node(id='4f41348daa2746c681394e844ab514a1', name='ChatPromptTemplate', da

In [8]:
chain.get_graph().print_ascii()

           +---------------------------------+         
           | Parallel<context,question>Input |         
           +---------------------------------+         
                    **               **                
                 ***                   ***             
               **                         **           
+----------------------+              +-------------+  
| VectorStoreRetriever |              | Passthrough |  
+----------------------+              +-------------+  
                    **               **                
                      ***         ***                  
                         **     **                     
           +----------------------------------+        
           | Parallel<context,question>Output |        
           +----------------------------------+        
                             *                         
                             *                         
                             *                  

In [9]:
chain.get_prompts()

[ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer the question based only on the following context:\n{context}\n\nQuestion: {question}\n'), additional_kwargs={})])]